# *Импорт библиотек*

In [ ]:
import gdown
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-white')
sns.set_context('notebook', rc={'figure.figsize': (10, 8)}, font_scale=1.5)

import numpy as np # Импортируем библиотеку numpy
import pandas as pd

import time
import random
import math

import cv2
from google.colab.patches import cv2_imshow
from PIL import Image, ImageFile, ImageEnhance, ImageFilter
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # print('Время обработки: {:.2f} с'.format(time.time() - self.t))
        seconds_total = time.time() - self.t
        minutes = int(seconds_total // 60)
        seconds = (seconds_total % 60) + (seconds_total - math.floor(seconds_total))
        print('Время обработки: {} мин {:.2f} c'.format(minutes, seconds))

with timex():
    print()

# Загрузка датафрейма для загрузки проверочных файлов и функции

In [ ]:
# Загрузка
gdown.download(f"https://drive.google.com/uc?id=1MvgckI85Uk2z-Q-HnQ7GXhacPZpKxGpF", 'df_info_new_ru.csv')
df = pd.read_csv('/content/df_info_new_ru.csv')

In [ ]:
df.columns

In [ ]:
df = df.sort_values(by='индекс файла')

In [ ]:
print(*df[df['проверка таблиц'] == 1].filename_new[:10], sep='\n')

In [ ]:
def get_img_path(base_path, filename):
    x = int(filename[:-4])
    lower_bound = (x // 100) * 100
    upper_bound = lower_bound + 100
    return base_path + f'{lower_bound}-{upper_bound}/' + filename

# Проверка
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
x_values = ['0001.png', '0019.png', '0100.png', '0654.png', '1119.png']

for x in x_values:
    y = get_img_path(base_path, x)
    print(y)

In [ ]:
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
test_filepaths = [get_img_path(base_path, i) for i in df.loc[(df['искажение документа'] == 1) & (df['искажение текста'] == 1)]['filename_new']]

In [ ]:
# Составляем выборки файлов для проверки таблиц
base_path = '/content/drive/MyDrive/ClearedReshapedBase/'
target_cols =  ['проверка таблиц', 'нестандартный контраст', 'нестандартная верхняя таблица', 'нестандартная нижняя таблица',
                'обрезанный текст', 'обрезанные границы таблиц', 'не белый фон', 'точечный текст', 'нестандартное форматирование текста',
                'плохая печать', 'искажение текста', 'искажение документа', '2 документа','2 банка в верхней таблице', 'грязь']

sample_dict = {col_name:[get_img_path(base_path, i) for i in list(df[df[col_name] == 1].filename_new)] for col_name in target_cols}

# Добавляем выборку без ошибок
rows_with_zeros = df[(df[df.columns[20:]] == 0).all(axis=1)]
filtered_df = df.loc[rows_with_zeros.index]
sample_dict['без ошибок'] = [get_img_path(base_path, f'{i:04}.png') for i in list(filtered_df['индекс файла'])[:100]]

In [ ]:
def print_img(image_path, figsize=(12,8), title=None):
    plt.rcParams['figure.facecolor'] = 'lightgray'
    img = Image.open(image_path)
    img = img.convert('RGB')

    # Отображение изображения с помощью Matplotlib
    plt.figure(figsize=figsize)
    plt.imshow(img)
    if title == None:
        plt.title(image_path[-8:])  # Установка заголовка из названия файла
    else:
        plt.title(title)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()

In [ ]:
for key, val in sample_dict.items():
    print(len(val),  '-', key)

In [ ]:
def print_img(img, figsize=(12,8), title='File'):
    # Отображение изображения с помощью Matplotlib
    plt.rcParams['figure.facecolor'] = 'lightgray'
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.title(title)  # Установка заголовка из названия файла
    plt.axis('off')  # Убрать оси координат
    plt.show()

# **PIL (ImageEnhance)**
> Функции из модуля ImageEnhance.
* Contrast (контраст)
* Brightness (яркость)
* Sharpness (резкость)

### *Из документации*

**PIL.ImageEnhance.Sharpness(Image)**

This class can be used to adjust the sharpness of an image. An enhancement factor of 0.0 gives a blurred image, a factor of 1.0 gives the original image, and a factor of 2.0 gives a sharpened image.

**PIL.ImageEnhance.Brightness(Image)**

This class can be used to control the brightness of an image. An enhancement factor of 0.0 gives a black image, a factor of 1.0 gives the original image, and greater values increase the brightness of the image.

**PIL.ImageEnhance.Contrast(Image)**

This class can be used to control the contrast of an image, similar to the contrast control on a TV set. An enhancement factor of 0.0 gives a solid grey image, a factor of 1.0 gives the original image, and greater values increase the contrast of the image.

### Тест разных значений
> в документации не написано максимальное и минимальное возможное значение

In [ ]:
def display_images_in_row(images, setting):
    fig, axs = plt.subplots(1, len(images), figsize=(18, 9))  # Создаем график с нужным количеством подграфиков в одном ряду

    for i, img in enumerate(images):
        axs[i].imshow(img)  # Отображаем изображение на соответствующем подграфике
        axs[i].axis('off')  # Убираем оси координат
        setting_name = ['Sharpness', 'Brightness', 'Contrast'][i]
        axs[i].set_title(f"{setting_name} - {setting}")  # Устанавливаем заголовок для каждого изображения

    plt.tight_layout()  # Для улучшения отображения графика
    plt.show()

In [ ]:
img_tmp = Image.open(sample_dict['без ошибок'][0]).convert('RGB')

img_lists = []
settings_list = [-1000, -500, -250,
                 -100, -50, -25, -10,
                 -5, -2.5, -2, -1.5,
                 -1, -0.75, -0.5, -0.25, -0.1,
                 0, 0.1, 0.25, 0.5, 0.75, 1,
                 1.25, 1.5, 1.75, 2, 2.5,
                 3, 5, 10, 25, 50, 100, 200,
                 500, 1000, 5000, 10000, 100000]

for factor in settings_list:
    img_list = []
    img_list.append(ImageEnhance.Sharpness(img_tmp).enhance(factor))
    img_list.append(ImageEnhance.Brightness(img_tmp).enhance(factor))
    img_list.append(ImageEnhance.Contrast(img_tmp).enhance(factor))
    img_lists.append(img_list)

In [ ]:
for i, sl in enumerate(settings_list):
    display_images_in_row(img_lists[i], sl)

## **Настройки для функций по результатам тестирования**

**Sharpness:**
* Значение 1 - не производит изменения.
* В негативных числах - работает и немного загрязняет изображение
* В положительных числах - повышает чёткость изображения

**Brightness:**
* Значение 1 - не производит изменения.
* В негативных числах - не работает, выдаёт полностью чёрное изображение.
* В положительных числах - повышает яркость изображения.

**Contrast:**
* Значение 1 - не производит изменения.
* В негативных числах - постепенно инвертирует изображение.
* В положительных числах - Увеличивает контраст изображений.

In [ ]:
# Функции
ImageEnhance.Sharpness(img).enhance(factor)
ImageEnhance.Brightness(img).enhance(factor)
ImageEnhance.Contrast(img).enhance(factor)

# Примерный предел фактора, в котором происходят изменения изображения (больше или меньше ставить смысла нет)
{'PIL_Sharpness':[-500,100],
 'PIL_Brightness':[0,100],
 'PIL_Contrast':[-100,5],}

# Диапазон, где изображение не ухудшается значительно для тестирования
{'PIL_Sharpness':[-10,100],
 'PIL_Brightness':[0,100],
 'PIL_Contrast':[0,5],}

# Все float

# **PIL (ImageFilter)**
> Фильтры для изображений из модуля ImageFilter.

> Простые фильтры:
* BLUR (размытие)
* DETAIL (детализация)
* EDGE_ENHANCE (четкость границ)
* EDGE_ENHANCE_MORE (улучшенная четкость границ)
* EMBOSS (тисненое изображение)
* SHARPEN (улучшение резкости)
* SMOOTH (сглаживание артефактов)
* SMOOTH_MORE (улучшенное сглаживание артефактов)

> Фильтры с настройками:
* BoxBlur (Размытие)
* GaussianBlur (Фильтр размытия по гауссу)
* UnsharpMask (Фильтр нерезкой маски)



## ***Простые фильтры***

In [ ]:
# Список фильтров
ImageFilter.BLUR
ImageFilter.DETAIL
ImageFilter.EDGE_ENHANCE
ImageFilter.EDGE_ENHANCE_MORE
ImageFilter.EMBOSS
ImageFilter.SHARPEN
ImageFilter.SMOOTH
ImageFilter.SMOOTH_MORE

# Нахождение контуров (можно попробовать использовать)
ImageFilter.CONTOUR
ImageFilter.FIND_EDGES

In [ ]:
# Использование
img = Image.open(sample_dict['без ошибок'][0]).convert('RGB')
filtered_img = img.filter(ImageFilter.EDGE_ENHANCE_MORE)            # Применение фильтра

print_img(img, (20,15), title='Orig')
print_img(filtered_img, (20,15), title='EDGE_ENHANCE_MORE')

## ***Фильтры с настройками***

### Описание фильтров

```
ImageFilter.BoxBlur(radius)
```

Класс ImageFilter.BoxBlur() размывает изображение, устанавливая для каждого пикселя среднее значение пикселей в квадратном поле, расширяющем радиус пикселей в каждом направлении. Поддерживает плавающий радиус произвольного размера. Использует оптимизированную реализацию, которая выполняется за линейное время относительно размера изображения для любого значения радиуса.

Аргумент radius - размер области в одном направлении. Радиус 0 не размывает и возвращает идентичное изображение. Радиус 1 занимает по 1 пикселю в каждом направлении, то есть всего 9 пикселей.

---
```
ImageFilter.GaussianBlur(radius)
```

размывает изображение с помощью последовательности расширенных прямоугольных фильтров, которые приближаются к ядру Гаусса.

Аргумент radius - стандартное отклонение ядра Гаусса.

---
```
ImageFilter.UnsharpMask(radius, percent, threshold)
```

Класс ImageFilter.UnsharpMask() представляет собой фильтр нерезкой маски.

Аргументы:

* radius: Радиус размытия
* percent: Нерезкая прочность, в процентах
* threshold: Порог определяет минимальное изменение яркости, которое будет увеличено.




### **Тесты настроек фильтров**

In [ ]:
def display_images_in_row(images, setting, setting_name='Unknown'):
    fig, axs = plt.subplots(1, len(images), figsize=(6*len(images), 9))  # Создаем график с нужным количеством подграфиков в одном ряду

    for i, img in enumerate(images):
        axs[i].imshow(img)  # Отображаем изображение на соответствующем подграфике
        axs[i].axis('off')  # Убираем оси координат
        axs[i].set_title(f"{setting_name} - {setting[i]}")  # Устанавливаем заголовок для каждого изображения

    plt.tight_layout()  # Для улучшения отображения графика
    plt.show()

#### **BoxBlur**
> Значение 0 - нет изменений (возвращает оригинальное изображение)

In [ ]:
img_tmp = Image.open(sample_dict['без ошибок'][0]).convert('RGB')

img_lists = [img_tmp]
settings_list = ['orig',-1,-0.5,0,0.5,1,2,5,8,10,20,50,100]

for s in settings_list[1:]:
    img_lists.append(img_tmp.filter(ImageFilter.BoxBlur(s)))

In [ ]:
display_images_in_row(img_lists, settings_list, 'BoxBlur')

#### **GaussianBlur**
> Значение 0 - нет изменений (возвращает оригинальное изображение)

In [ ]:
img_tmp = Image.open(sample_dict['без ошибок'][0]).convert('RGB')

img_lists = [img_tmp]
settings_list = ['orig',-1,-0.5,0,0.5,1,2,5,8,10,20,50,100]

for s in settings_list[1:]:
    img_lists.append(img_tmp.filter(ImageFilter.GaussianBlur(s)))

In [ ]:
display_images_in_row(img_lists, settings_list, 'GaussianBlur')

#### **UnsharpMask**
> Не видно работы метода

In [ ]:
# Можно изменить 3 параметра
# Стандартные настройки
ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3)

##### *Параметр radius*

In [ ]:
img_tmp = Image.open(sample_dict['без ошибок'][0]).convert('RGB')

img_lists = [img_tmp]
settings_list = ['orig',-1,-0.5,0,0.5,1,2,5,8,10,20,50,100]

for val in settings_list[1:]:
    img_lists.append(img_tmp.filter(ImageFilter.UnsharpMask(radius=val, percent=150, threshold=3)))

In [ ]:
display_images_in_row(img_lists, settings_list, 'GaussianBlur radius')

##### *Параметр percent*

In [ ]:
img_tmp = Image.open(sample_dict['без ошибок'][0]).convert('RGB')

img_lists = [img_tmp]
settings_list = ['orig',0,25,50,100,125,150,300,500,1000,1500,2500,10000]

for val in settings_list[1:]:
    img_lists.append(img_tmp.filter(ImageFilter.UnsharpMask(radius=2, percent=val, threshold=3)))

In [ ]:
display_images_in_row(img_lists, settings_list, 'GaussianBlur percent')

##### *Параметр threshold*

In [ ]:
img_tmp = Image.open(sample_dict['без ошибок'][0]).convert('RGB')

img_lists = [img_tmp]
settings_list = ['orig',-1,0,1,2,5,8,10,20,50,100,150,200,250,1000]

for val in settings_list[1:]:
    img_lists.append(img_tmp.filter(ImageFilter.UnsharpMask(radius=10, percent=500, threshold=val)))

In [ ]:
display_images_in_row(img_lists, settings_list, 'GaussianBlur threshold')

### **Настройки для фильтров по результатам тестирования**

In [ ]:
# Настройки для фильтров
{'PIL_BoxBlur':[0,10],                  # float
 'PIL_GaussianBlur':[0,5],              # float
}

#   'PIL_UnsharpMask_radius':[],
#   'PIL_UnsharpMask_percent':[],
#   'PIL_UnsharpMask_threshold':[],

### Кастомные фильтры
> Такие есть, но их долго тестировать и для теста нужно много данных генерировать

Класс ImageFilter.Kernel() создает ядро ​​свертки. Поддерживается только ядра 3x3 и 5x5 для целых чисел и ядер с плавающей запятой. Можно применять только к изображениям L и RGB.

Аргументы:

* size: Размер ядра в виде (width, height). Должно быть (3,3) или (5,5).
* kernel: Последовательность, содержащая веса ядра.
* scale: Масштаб. Если задано, то результат для каждого пикселя делится на это значение. По умолчанию используется сумма весов ядра.
* offset: Смещение. Если задано, это значение добавляется к результату после того, как оно было разделено на scale.

In [ ]:
ImageFilter.Kernel(size, kernel, scale=None, offset=0)

from PIL import ImageGrab, ImageFilter

# создание скриншота
img_orig = ImageGrab.grab()
img_orig.save('test_orig.png')

# применяем фильтр `ImageFilter.Kernel`
img = img_orig.filter(ImageFilter.Kernel((3, 3),
       (-1, -1, -1, -1, 9, -1, -1, -1, -1), 1, 0))
# сохраним для сравнения
img.save('test_Filter_Kernel_9.png')

# с другими весами ядра
img = img_orig.filter(ImageFilter.Kernel((3, 3),
       (-1, -1, -1, -1, 11, -2, -2, -2, -2), 1, 0))
img.save('test_Filter_Kernel_11.png')

Класс ImageFilter.RankFilter() создает ранговый фильтр. Ранговый фильтр сортирует все пиксели в окне заданного размера и возвращает ранговое значение.

Аргументы:

* size: Размер ядра, в пикселях.
* rank: Какое значение пикселя выбрать. Используйте 0 для минимального фильтра, для медианного фильтра size * size / 2, для максимального фильтра size * size - 1 и т. д.

In [ ]:
ImageFilter.RankFilter(size, rank)

# **cv2**
> Функции:
* dilate (Расширение)
* erode (Размывание)
* GaussianBlur (Фильтр размытия по гауссу)
* normalize (нормализация)
* threshold (Пороговое значение)
* adaptiveThreshold (Адаптивное пороговое значение)
* fastNlMeansDenoising (Уменьшение шумов)

In [ ]:
def display_images_in_row(images, setting, setting_name='Unknown', cv2_type=True):
    if cv2_type == True:
        images = [cv2.cvtColor(i, cv2.COLOR_GRAY2RGB) for i in images]

    fig, axs = plt.subplots(1, len(images), figsize=(6*len(images), 9))  # Создаем график с нужным количеством подграфиков в одном ряду

    for i, img in enumerate(images):
        axs[i].imshow(img)  # Отображаем изображение на соответствующем подграфике
        axs[i].axis('off')  # Убираем оси координат
        axs[i].set_title(f"{setting_name} - {setting[i]}")  # Устанавливаем заголовок для каждого изображения

    plt.tight_layout()  # Для улучшения отображения графика
    plt.show()

img = cv2.imread(sample_dict['без ошибок'][0], cv2.IMREAD_UNCHANGED)

In [ ]:
img.shape

## **thresholding**

In [ ]:
print(cv2.threshold.__doc__)

In [ ]:
# enum ThresholdTypes
# {
#     THRESH_BINARY     = 0,
#     THRESH_BINARY_INV = 1,
#     THRESH_TRUNC      = 2,
#     THRESH_TOZERO     = 3,
#     THRESH_TOZERO_INV = 4,
#     THRESH_MASK       = 7,
#     THRESH_OTSU       = 8,
#     THRESH_TRIANGLE   = 16,
# };

In [ ]:
cv2.threshold(img, thresh=50, maxval=255, type=cv2.THRESH_BINARY + cv2.THRESH_OTSU)

##### *Параметр thresh*

In [ ]:
img_lists = [img]
settings_list = ['orig', 0,5,10,25,50,100,150,200,250,255]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.threshold(img, thresh=val, maxval=255, type=cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1])
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding thresh')

##### *Параметр maxval*

In [ ]:
img_lists = [img]
settings_list = ['orig', 0,5,10,25,50,100,150,200,250,255]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.threshold(img, thresh=0, maxval=val, type=cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1])
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding maxval')

##### *Параметр type*

In [ ]:
img_lists = [img]
settings_list = ['orig'] + [i for i in range(17)]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.threshold(img, thresh=0, maxval=255, type=val)[1])
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding type')

In [ ]:
img_lists = [img]
settings_list = ['orig'] + [i for i in range(17)]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.threshold(img, thresh=50, maxval=200, type=val)[1])
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding type')

In [ ]:
img_lists = [img]
settings_list = ['orig'] + [i for i in range(17)]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.threshold(img, thresh=255, maxval=0, type=val)[1])
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding type')

## **adaptiveThreshold**

In [ ]:
print(cv2.adaptiveThreshold.__doc__)

In [ ]:
cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 31, 10)

In [ ]:
cv2.adaptiveThreshold(img, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=31, C=10)

##### *Параметр maxValue*

In [ ]:
img_lists = [img]
settings_list = ['orig', 0,5,10,25,50,100,150,200,250,255]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.adaptiveThreshold(img, maxValue=val, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=31, C=10))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding maxValue')

##### *Параметр adaptiveMethod*

In [ ]:
img_lists = [img]
settings_list = ['orig', cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.ADAPTIVE_THRESH_MEAN_C]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.adaptiveThreshold(img, maxValue=255, adaptiveMethod=val, thresholdType=cv2.THRESH_BINARY, blockSize=31, C=10))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding adaptiveMethod')

##### *Параметр thresholdType*

In [ ]:
img_lists = [img]
settings_list = ['orig'] + [0,1]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.adaptiveThreshold(img, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=val, blockSize=31, C=10))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding thresholdType')

##### *Параметр blockSize*

In [ ]:
img_lists = [img]
settings_list = ['orig',3,5,11,21,31,41,51,101,251,501,1001]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.adaptiveThreshold(img, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=val, C=10))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding blockSize')

##### *Параметр C*

In [ ]:
img_lists = [img]
settings_list = ['orig', 1,5,10,20,50,100,150,175,200,225,250]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.adaptiveThreshold(img, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=31, C=val))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'thresholding C')

## **normalize**

In [ ]:
print(cv2.normalize.__doc__)

In [ ]:
cv2.normalize(img, np.zeros((img.shape[0], img.shape[1])), 0, 255, cv2.NORM_MINMAX)

In [ ]:
cv2.normalize(img, dst=np.zeros((img.shape[0], img.shape[1])), alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

##### *Параметр alpha*

In [ ]:
img_lists = [img]
settings_list = ['orig', 0, 5, 10, 25, 50, 75, 100, 150, 200, 225, 250]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.normalize(img, dst=np.zeros((img.shape[0], img.shape[1])), alpha=val, beta=255, norm_type=cv2.NORM_MINMAX))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'normalize alpha')

##### *Параметр beta*

In [ ]:
img_lists = [img]
settings_list = ['orig', 0, 5, 10, 25, 50, 75, 100, 150, 200, 225, 250]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.normalize(img, dst=np.zeros((img.shape[0], img.shape[1])), alpha=0, beta=val, norm_type=cv2.NORM_MINMAX))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'normalize beta')

##### *Параметр norm_type*

In [ ]:
img_lists = [img]
settings_list = ['orig', cv2.NORM_INF, cv2.NORM_L1, cv2.NORM_L2,cv2.NORM_MINMAX]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.normalize(img, dst=np.zeros((img.shape[0], img.shape[1])), alpha=0, beta=255, norm_type=val))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'normalize norm_type')

## **dilate**

In [ ]:
print(cv2.dilate.__doc__)

In [ ]:
cv2.dilate(img, kernel=(10, 10), iterations=3, anchor=(-1,-1))

##### *Параметр kernel*

In [ ]:
img_lists = [img]
settings_list = ['orig', (1,1), (3,3), (5,5), (8,8), (11,11), (21,21), (51,51), (100,100),
                 (3,5), (3,11), (3,21), (3,51),
                 (5,3), (11,3), (21,3), (51,3),]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.dilate(img, kernel=val, iterations=3, anchor=(-1,-1)))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'dilate kernel')

##### *Параметр anchor*

In [ ]:
img_lists = [img]
settings_list = ['orig', (0,0), (-1,0), (0,-1), (-1,-1)]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.dilate(img, kernel=(11,11), iterations=3, anchor=(-1,-1)))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'dilate kernel')

##### *Параметр iterations*

In [ ]:
img_lists = [img]
settings_list = ['orig', 1,2,3,5,10,20,50,100,200,1000]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.dilate(img, kernel=(10,10), iterations=val, anchor=(-1,-1)))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'dilate iterations')

##### *Параметр BorderType*

In [ ]:
img_lists = [img]
settings_list = ['orig', cv2.BORDER_CONSTANT, cv2.BORDER_REPLICATE,
                 cv2.BORDER_REFLECT, cv2.BORDER_REFLECT_101, cv2.BORDER_ISOLATED]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.dilate(img, kernel=(10, 10), iterations=3, anchor=(-1,-1), borderType=val))
    print('-----------------')

img_lists = [i for i in img_lists if type(i) != None]

display_images_in_row(img_lists, settings_list, 'GaussianBlur borderType')

## **erode**

In [ ]:
print(cv2.erode.__doc__)

In [ ]:
cv2.erode(img, kernel=(10, 10), iterations=3, anchor=(-1,-1))

##### *Параметр kernel*

In [ ]:
img_lists = [img]
settings_list = ['orig', (1,1), (3,3), (5,5), (8,8), (11,11), (21,21), (51,51), (100,100),
                 (3,5), (3,11), (3,21), (3,51),
                 (5,3), (11,3), (21,3), (51,3),]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.erode(img, kernel=val, iterations=3, anchor=(-1,-1)))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'erode kernel')

##### *Параметр anchor*

In [ ]:
img_lists = [img]
settings_list = ['orig', (0,0), (-1,0), (0,-1), (-1,-1)]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.erode(img, kernel=(11,11), iterations=3, anchor=(-1,-1)))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'erode anchor')

##### *Параметр iterations*

In [ ]:
img_lists = [img]
settings_list = ['orig', 1,2,3,5,10,20,50,100,200,1000]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.erode(img, kernel=(10,10), iterations=val, anchor=(-1,-1)))
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'erode iterations')

##### *Параметр BorderType*

In [ ]:
print([cv2.BORDER_CONSTANT, cv2.BORDER_REPLICATE,cv2.BORDER_REFLECT, cv2.BORDER_REFLECT_101, cv2.BORDER_ISOLATED])

In [ ]:
img_lists = [img]
settings_list = ['orig', cv2.BORDER_CONSTANT, cv2.BORDER_REPLICATE,
                 cv2.BORDER_REFLECT, cv2.BORDER_REFLECT_101, cv2.BORDER_ISOLATED]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.erode(img, kernel=(10, 10), iterations=3, anchor=(-1,-1), borderType=val))
    print('-----------------')

img_lists = [i for i in img_lists if type(i) != None]

display_images_in_row(img_lists, settings_list, 'erode borderType')

## **GaussianBlur**

In [ ]:
print(cv2.GaussianBlur.__doc__)

In [ ]:
cv2.GaussianBlur(img, ksize=(11, 11), sigmaX=0, sigmaY=0)

##### *Параметр ksize*

In [ ]:
img_lists = [img]
settings_list = ['orig', (1,1), (3,3), (5,5), (11,11), (21,21), (51,51),
                 (3,5), (3,11), (3,21), (3,51),
                 (5,3), (11,3), (21,3), (51,3),]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.GaussianBlur(img, ksize=val, sigmaX=0, sigmaY=0)) # normType=cv2.NORM_L2
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'GaussianBlur ksize')

##### *Параметр sigmaX*

In [ ]:
img_lists = [img]
settings_list = ['orig', 1,2,3,5,10,20,50,100,200,500]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.GaussianBlur(img, ksize=(11,11), sigmaX=val, sigmaY=0)) # normType=cv2.NORM_L2
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'GaussianBlur sigmaX')

##### *Параметр sigmaY*

In [ ]:
img_lists = [img]
settings_list = ['orig', 1,2,3,5,10,20,50,100,200,500]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.GaussianBlur(img, ksize=(11,11), sigmaX=0, sigmaY=val)) # normType=cv2.NORM_L2
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'GaussianBlur sigmaY')

##### *Параметр BorderType*

In [ ]:
img_lists = [img]
settings_list = ['orig', cv2.BORDER_CONSTANT, cv2.BORDER_REPLICATE, cv2.BORDER_WRAP,
                 cv2.BORDER_REFLECT, cv2.BORDER_REFLECT_101, cv2.BORDER_ISOLATED]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.GaussianBlur(img, ksize=(11,11), sigmaX=0, sigmaY=0, borderType=val)) # normType=cv2.NORM_L2
    print('-----------------')

display_images_in_row(img_lists, settings_list, 'GaussianBlur borderType')

## **fastNlMeansDenoising**
> Долго работает

In [ ]:
print(cv2.fastNlMeansDenoising.__doc__)

In [ ]:
# Стандартные настройки
cv2.fastNlMeansDenoising(img, h=3, templateWindowSize=7, searchWindowSize=21)

##### *Параметр templateWindowSize*

In [ ]:
img_lists = [img]
settings_list = ['orig',3,5,7,10,15,25,50,100]

for val in settings_list[1:]:
    img_lists.append(cv2.fastNlMeansDenoising(img, h=3, templateWindowSize=val, searchWindowSize=21)) # normType=cv2.NORM_L2

display_images_in_row(img_lists, settings_list, 'fastNlMeansDenoising templateWindowSize')

##### *Параметр h*

In [ ]:
img_lists = [img]
settings_list = ['orig',1,2,3,4,5,7,10,15,25,50,100]

for val in settings_list[1:]:
    img_lists.append(cv2.fastNlMeansDenoising(img, h=val, templateWindowSize=7, searchWindowSize=21))

display_images_in_row(img_lists, settings_list, 'fastNlMeansDenoising h')

##### *Параметр searchWindowSize*

In [ ]:
img_lists = [img]
settings_list = ['orig',3,5,10,15,21,30,50]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.fastNlMeansDenoising(img, h=3, templateWindowSize=7, searchWindowSize=val)) # normType=cv2.NORM_L2
    print()

display_images_in_row(img_lists, settings_list, 'fastNlMeansDenoising searchWindowSize')

##### *Доп тесты параметров*

In [ ]:
img_lists = [img]
settings_list = ['orig',3,5,7,10,15,25,50,100]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.fastNlMeansDenoising(img, h=val, templateWindowSize=5, searchWindowSize=5)) # normType=cv2.NORM_L2
    print()

display_images_in_row(img_lists, settings_list, 'fastNlMeansDenoising h')

In [ ]:
img_lists = [img]
settings_list = ['orig',3,5,7,10,15,25,50,100]

for val in settings_list[1:]:
    print(val)
    with timex():
        img_lists.append(cv2.fastNlMeansDenoising(img, h=3, templateWindowSize=val, searchWindowSize=5)) # normType=cv2.NORM_L2
    print()

display_images_in_row(img_lists, settings_list, 'fastNlMeansDenoising templateWindowSize')

## **Настройки для функций по результатам тестирования**

In [ ]:
{'cv2_fastNlMeansDenoising_templateWindowSize':[0,100],     # только целые числа
 'cv2_fastNlMeansDenoising_searchWindowSize':[0,21],        # Если большое значение, то сильно увеличивается время обработки
 'cv2_fastNlMeansDenoising_h':[0,100],

 'cv2_GaussianBlur_ksize_x':[1,51], # Только нечётные целые числа
 'cv2_GaussianBlur_ksize_y':[1,51], # Только нечётные целые числа
 'cv2_GaussianBlur_sigmaX':[0,10],
 'cv2_GaussianBlur_sigmaY':[0,10],
 'cv2_GaussianBlur_borderType':[0,1,2,3,4,16], # Выбор из списка

 'cv2_erode_kernel_x':[1,51],               # только целые числа
 'cv2_erode_kernel_y':[1,51],               # только целые числа
 'cv2_erode_anchor':[(0,0), (-1,0), (0,-1), (-1,-1)], # Выбор из списка
 'cv2_erode_iterations':[0,20],             # только целые числа
 'cv2_erode_BorderType':[0, 1, 2, 4, 16],   # Выбор из списка

 'cv2_dilate_kernel_x':[1,51],              # только целые числа
 'cv2_dilate_kernel_y':[1,51],              # только целые числа
 'cv2_dilate_anchor':[(0,0), (-1,0), (0,-1), (-1,-1)], # Выбор из списка
 'cv2_dilate_iterations':[0,20],            # только целые числа
 'cv2_dilate_BorderType':[0, 1, 2, 4, 16],  # Выбор из списка

 'cv2_normalize_alpha':[0,255], # только целые числа
 'cv2_normalize_beta':[0,255],  # только целые числа
 'cv2_normalize_norm_type':[1, 2, 4, 32], # Выбор из списка

 'cv2_thresholding_thresh':[0,255], # только целые числа
 'cv2_thresholding_maxval':[0,255], # только целые числа
 'cv2_thresholding_type':[i for i in range(17)], # Выбор из списка / или только целые числа в диапазоне [0,16]

 'cv2_adaptiveThreshold_maxValue':[0,255],  # только целые числа
 'cv2_adaptiveThreshold_adaptiveMethod':[0,1], # Выбор из списка
 'cv2_adaptiveThreshold_thresholdType':[0,1], # Выбор из списка
 'cv2_adaptiveThreshold_blockSize':[3,101], # только нечётные целые числа больше или равные 3
 'cv2_adaptiveThreshold_C':[0,255],         # только целые числа

 }

# **Все настройки для всех методов (cv2, PIL)**

In [ ]:
################################################################################
#                               cv2                                            #
################################################################################

# Функции
cv2.threshold(img, thresh=50, maxval=255, type=cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.adaptiveThreshold(img, maxValue=255, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType=cv2.THRESH_BINARY, blockSize=31, C=10)
cv2.normalize(img, dst=np.zeros((img.shape[0], img.shape[1])), alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
cv2.dilate(img, kernel=(10, 10), iterations=3, anchor=(-1,-1))
cv2.erode(img, kernel=(10, 10), iterations=3, anchor=(-1,-1))
cv2.GaussianBlur(img, ksize=(11, 11), sigmaX=0, sigmaY=0)
cv2.fastNlMeansDenoising(img, h=3, templateWindowSize=7, searchWindowSize=21)

# Настройки параметров
# Если написано только 2 числа в списке - [0,10], то это значит, что для этого
# параметра подходит любое число (float) в данном диапазоне.
# Остальные пояснения, думаю, понятны.

{'cv2_fastNlMeansDenoising_templateWindowSize':[0,100],     # только целые числа
 'cv2_fastNlMeansDenoising_searchWindowSize':[0,21],        # Если большое значение, то сильно увеличивается время обработки
 'cv2_fastNlMeansDenoising_h':[0,100],

 'cv2_GaussianBlur_ksize_x':[1,51], # Только нечётные целые числа
 'cv2_GaussianBlur_ksize_y':[1,51], # Только нечётные целые числа
 'cv2_GaussianBlur_sigmaX':[0,10],
 'cv2_GaussianBlur_sigmaY':[0,10],
 'cv2_GaussianBlur_borderType':[0,1,2,3,4,16], # Выбор из списка

 'cv2_erode_kernel_x':[1,51],               # только целые числа
 'cv2_erode_kernel_y':[1,51],               # только целые числа
 'cv2_erode_anchor':[(0,0), (-1,0), (0,-1), (-1,-1)], # Выбор из списка
 'cv2_erode_iterations':[0,20],             # только целые числа
 'cv2_erode_BorderType':[0, 1, 2, 4, 16],   # Выбор из списка

 'cv2_dilate_kernel_x':[1,51],              # только целые числа
 'cv2_dilate_kernel_y':[1,51],              # только целые числа
 'cv2_dilate_anchor':[(0,0), (-1,0), (0,-1), (-1,-1)], # Выбор из списка
 'cv2_dilate_iterations':[0,20],            # только целые числа
 'cv2_dilate_BorderType':[0, 1, 2, 4, 16],  # Выбор из списка

 'cv2_normalize_alpha':[0,255], # только целые числа
 'cv2_normalize_beta':[0,255],  # только целые числа
 'cv2_normalize_norm_type':[1, 2, 4, 32], # Выбор из списка

 'cv2_thresholding_thresh':[0,255], # только целые числа
 'cv2_thresholding_maxval':[0,255], # только целые числа
 'cv2_thresholding_type':[i for i in range(17)], # Выбор из списка / или только целые числа в диапазоне [0,16]

 'cv2_adaptiveThreshold_maxValue':[0,255],  # только целые числа
 'cv2_adaptiveThreshold_adaptiveMethod':[0,1], # Выбор из списка
 'cv2_adaptiveThreshold_thresholdType':[0,1], # Выбор из списка
 'cv2_adaptiveThreshold_blockSize':[3,101], # только нечётные целые числа больше или равные 3
 'cv2_adaptiveThreshold_C':[0,255],         # только целые числа
 }

################################################################################
#                               PIL                                            #
################################################################################

# Функции
ImageEnhance.Sharpness(img).enhance(factor)
ImageEnhance.Brightness(img).enhance(factor)
ImageEnhance.Contrast(img).enhance(factor)

# Список простых фильтров
ImageFilter.BLUR
ImageFilter.DETAIL
ImageFilter.EDGE_ENHANCE
ImageFilter.EDGE_ENHANCE_MORE
ImageFilter.EMBOSS
ImageFilter.SHARPEN
ImageFilter.SMOOTH
ImageFilter.SMOOTH_MORE

# Нахождение контуров (можно попробовать использовать)
ImageFilter.CONTOUR
ImageFilter.FIND_EDGES

# Список фильтров с параметром
ImageFilter.BoxBlur(factor)
ImageFilter.GaussianBlur(factor)

# Использование фильтров
img.filter(ImageFilter.EDGE_ENHANCE_MORE)
img.filter(ImageFilter.BoxBlur(factor))
img.filter(ImageFilter.GaussianBlur(factor))

# Настройки для фильтров
{'PIL_BoxBlur':[0,10],
 'PIL_GaussianBlur':[0,5],}

# Примерный предел фактора, в котором происходят изменения изображения (больше или меньше ставить смысла нет)
{'PIL_Sharpness':[-500,100],
 'PIL_Brightness':[0,100],
 'PIL_Contrast':[-100,5],}

# Диапазон, где изображение не ухудшается значительно для тестирования
{'PIL_Sharpness':[-10,100],
 'PIL_Brightness':[0,100],
 'PIL_Contrast':[0,5],}

In [ ]:
# Полезно для перевода из PIL в cv2
def to_numpy(im):
    im.load()
    # unpack data
    e = Image._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

# Открыть файл cv2
img_cv2 = cv2.imread('путь_к_файлу.png', cv2.IMREAD_UNCHANGED)

# Открыть файл PIL
img_PIL = Image.open('путь_к_файлу.png').convert('RGB')

# Быстрый перевод изображения PIL в массив для использования библиотеки cv2
img_cv2 = to_numpy(img_PIL)